# Getting started with LLM APIs

This Notebook has a code snippet that lets us call different LLM APIs using just one set of code.

While each LLM has its set of config, here's a cool perk of using [`txtai`](https://neuml.github.io/txtai/): plus [`litellm`](https://docs.litellm.ai/docs/) working behind the scenes. The big win with `txtai` is that it helps standardize inputs and outputs across several LLMs. **This means we can seamlessly call any LLM API**.

In this Notebook, you can use the same code to call _**Gemini**_, _**VertexAI**_, _**Mistral**_, _**Cohere**_, _**AWS Bedrock**_, _**OpenAI**_, _**Claude by Anthropic**_, _**Groq**_, _and more_!

Isn't that amazing? One code for many different LLM APIs - **that's the beauty of these libraries**!

## Install dependencies

This session should guide you through all the essential dependencies you'll need to run the Python script for various LLM APIs.

You can choose the dependencies based on your specific needs. Each dependency is carefully commented on in the code, so you'll know exactly which API it supports.

The core dependencies you'll typically need are: `txtai` and `txtai[pipeline]`.

In [ ]:
%%capture
!pip install txtai==8.1.0
!pip install txtai[pipeline]

# to use Vertex AI
# https://github.com/BerriAI/litellm/issues/5483
!pip install google-cloud-aiplatform==1.75.0

# to use AWS Bedrock
# https://docs.litellm.ai/docs/providers/bedrock
!pip install boto3==1.35.88

- _**A friendly reminder**: things can change unexpectedly in the ever-evolving world of coding. One day, your code works flawlessly; the next, it might throw a tantrum for no apparent reason. That's why it's essential to specify the dependencies' versions using the latest available versions when writing the code._

## LLM API Configuration

This session is like a special Python dictionary that holds all the cool configurations for our LLM AI model. It includes details like environment variables, the model's name, text embedding parameters, and other fancy settings.

One important key here is `IS_ENABLED`. When it's set to `True`, it's like giving the model a green light to shine! But if you ever feel like taking a break or don't need it for a while, you can easily set this key to `False`, and the model will chill out.

In [62]:
import os, getpass
from txtai import LLM, Embeddings

# https://neuml.github.io/txtai/install/
# https://neuml.github.io/txtai/pipeline/text/llm/#example
# https://neuml.github.io/txtai/embeddings/configuration/vectors/#method
# https://docs.litellm.ai/docs/embedding/supported_embedding
LLM_MODEL_CONFIG = {
  'GEMINI': {
    'IS_ENABLED': True,
    'ENV_VAR': ['GEMINI_API_KEY'],
    # https://docs.litellm.ai/docs/providers/gemini
    # https://ai.google.dev/gemini-api/docs/models/gemini
    'LLM_MODEL_NAME': 'gemini/gemini-pro',
    # https://github.com/BerriAI/litellm/tree/12c4e7e695edb07d403dd14fc768a736638bd3d1/litellm/llms/vertex_ai
    # https://github.com/BerriAI/litellm/blob/e19bb55e3b4c6a858b6e364302ebbf6633a51de5/model_prices_and_context_window.json#L2625
    'TEXT_EMBEDDING_PATH': 'gemini/text-embedding-004'
  },
  'COHERE': {
    'IS_ENABLED': False,
    'ENV_VAR': ['COHERE_API_KEY'],
    # https://docs.litellm.ai/docs/providers/cohere
    'LLM_MODEL_NAME': 'command-r',
    'TEXT_EMBEDDING_PATH': 'cohere/embed-english-v3.0'
  },
  'AWS_BEDROCK': {
    'IS_ENABLED': False,
    'ENV_VAR': ['AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'AWS_REGION_NAME'],
    # https://docs.litellm.ai/docs/providers/bedrock
    'LLM_MODEL_NAME': 'bedrock/amazon.titan-text-lite-v1',
    'TEXT_EMBEDDING_PATH': 'amazon.titan-embed-text-v1'
  },
  'MISTRAL': {
    'IS_ENABLED': False,
    'ENV_VAR': ['MISTRAL_API_KEY'],
    # https://docs.litellm.ai/docs/providers/mistral
    'LLM_MODEL_NAME': 'mistral/mistral-tiny',
    'TEXT_EMBEDDING_PATH': 'mistral/mistral-embed'
  },
  'VERTEXAI': {
    'IS_ENABLED': False,
    'ENV_VAR': ['GOOGLE_APPLICATION_CREDENTIALS', 'GOOGLE_VERTEX_PROJECT', 'GOOGLE_VERTEX_LOCATION'],
    'ENV_VAR_SETUP': None,
    # https://docs.litellm.ai/docs/providers/vertex
    'LLM_MODEL_NAME': 'vertex_ai/gemini-pro',
    'TEXT_EMBEDDING_PATH': 'vertex_ai/text-embedding-004'
  },
  'GROQ': {
    'IS_ENABLED': False,
    'ENV_VAR': ['GROQ_API_KEY'],
    # https://docs.litellm.ai/docs/providers/groq
    # https://console.groq.com/docs/models
    'LLM_MODEL_NAME': 'groq/llama3-8b-8192',
    # https://groq.com/retrieval-augmented-generation-with-groq-api/
    'TEXT_EMBEDDING_PATH': 'jinaai/jina-embeddings-v2-base-en',
  },
  'OPENAI': {
    'IS_ENABLED': False,
    'ENV_VAR': ['OPENAI_API_KEY'],
    # https://docs.litellm.ai/docs/providers/openai
    # https://platform.openai.com/docs/models
    'LLM_MODEL_NAME': 'gpt-4o-mini-2024-07-18',
    'TEXT_EMBEDDING_PATH': 'text-embedding-3-small',
  },
  'CLAUDE': {
    'IS_ENABLED': False,
    'ENV_VAR': ['ANTHROPIC_API_KEY', 'VOYAGE_API_KEY'],
    # https://docs.litellm.ai/docs/providers/anthropic
    # https://docs.anthropic.com/en/docs/about-claude/models
    'LLM_MODEL_NAME': 'anthropic/claude-3-5-haiku-20241022',
    # https://docs.anthropic.com/en/docs/build-with-claude/embeddings
    'TEXT_EMBEDDING_PATH': 'voyage/voyage-01',
  }
}

import litellm
# # https://github.com/BerriAI/litellm/blob/11932d0576a073d83f38a418cbdf6b2d8d4ff46f/litellm/litellm_core_utils/get_llm_provider_logic.py#L322
litellm.suppress_debug_info = True
# https://docs.litellm.ai/docs/debugging/local_debugging#set-verbose
litellm.set_verbose=False

def custom_setup ():
  def vertexai_custom_setup ():
    # https://docs.litellm.ai/docs/embedding/supported_embedding#usage---embedding
    # https://docs.litellm.ai/docs/providers/vertex
    litellm.vertex_project = os.environ['GOOGLE_VERTEX_PROJECT']
    litellm.vertex_location = os.environ['GOOGLE_VERTEX_LOCATION']
  LLM_MODEL_CONFIG['VERTEXAI']['ENV_VAR_SETUP'] = vertexai_custom_setup
custom_setup()

LLM_MODELS = {k: v for k, v in LLM_MODEL_CONFIG.items() if v['IS_ENABLED']}
ENV_VARS = [v['ENV_VAR'] for k, v in LLM_MODELS.items()]
ENV_VARS = [x for xs in ENV_VARS for x in xs] # flatten array

print("config set!")

config set!


### Vertex AI

When working with VertexAI in a Jupyter notebook, an environment variable called `GOOGLE_APPLICATION_CREDENTIALS` points to a credentials file. This variable is like giving VertexAI a secret map to find the JSON file containing your service account key.

Imagine you've set `GOOGLE_APPLICATION_CREDENTIALS` to `application_default_credentials.json`. The service account key is stored in a file with that exact name. And guess what? This file must be in the same place you're working (your current working directory).

Here's how your directory might look:
```
.
├── ..
├── sample_data/
└── application_default_credentials.json
```

If you use Google Colab, your directory structure will differ slightly, but don't worry! It'll look something like this:

```
.
├── bin
├── boot
├── content/
│   ├── sample_data/
│   └── application_default_credentials.json
├── datalab
├── dev
├── etc
├── home
├── lib
├── lib32
└── lib64
```


## Environment Variables

This session has scripts to reset or set environment variables _(env vars)_. Using env vars is a great way to keep those sensitive API KEY values safe and sound, away from unwanted eyes.

### Script to Reset Environment Variables

There are a bunch of reasons to reset the environment variable here. Let's go over a few:
- **Switching API Keys:** Sometimes, we need to change to a different API key.
- **Typos Happen:** We're all human, and typos can sneak in!
- **Skipping Variables:** Sometimes we need to skip one or another, so we set the env var to empty.
- **Fresh Start:** After tweaking a script or setup, it's always good to rerun with a fresh environment.


In [59]:
if 'MISTRAL_API_KEY' in os.environ:
  del os.environ['MISTRAL_API_KEY']

In [ ]:
for ENV_VARS in [v['ENV_VAR'] for k, v in LLM_MODEL_CONFIG.items()]:
  for ENV_VAR in ENV_VARS:
    if ENV_VAR in os.environ:
      del os.environ[ENV_VAR]
    print(f'unset env var: {ENV_VAR}')


unset env var: GEMINI_API_KEY
unset env var: COHERE_API_KEY
unset env var: AWS_ACCESS_KEY_ID
unset env var: AWS_SECRET_ACCESS_KEY
unset env var: AWS_REGION_NAME
unset env var: MISTRAL_API_KEY
unset env var: GOOGLE_APPLICATION_CREDENTIALS
unset env var: GOOGLE_VERTEX_PROJECT
unset env var: GOOGLE_VERTEX_LOCATION


### Script to Set Environment Variables

In this session, the script will stroll through the config dictionary to cherry-pick only the enabled LLM models.

Once your favorites have been gathered, the script will prompt you to set all the necessary environment variables.

The more LLM models you have enabled, the more prompts you'll see - but don't worry, each environment variable will prompt only once.

In [63]:
for ENV_VAR in ENV_VARS:
  os.environ[ENV_VAR] = getpass.getpass(f"enter {ENV_VAR}: ") if not ENV_VAR in os.environ else os.environ[ENV_VAR]

LLM_MODELS_WITH_CUSTOM_SETUP = {k: v for k, v in LLM_MODELS.items() if 'ENV_VAR_SETUP' in v}
for LLM_MODEL, LLM_CONFIG in LLM_MODELS_WITH_CUSTOM_SETUP.items():
  print(f"running custom env var setup for {LLM_MODEL}...")
  LLM_CONFIG['ENV_VAR_SETUP']()
  print(f"done setup for {LLM_MODEL}")

print("all env var set!")

running custom env var setup for VERTEXAI...
done setup for VERTEXAI
all env var set!


# Code Snippet

You can run the code snippet after installing the necessary dependencies and setting up the required environment variables!




## Introduction

This code snippet is designed to achieve only two tasks: _(i)_ run an LLM Pipeline; _(ii)_ text Embed using the LLM Model.

The code snippet has two tasks:
1. **Running an LLM Pipeline**:
   - To kickstart an LLM pipeline, you'll need a prompt and a model name. It's as simple as that!
2. **Text Embedding using the LLM Model**:
   - To embed text with the LLM model, you'll need some text and the name of the text embedding model.

In this script, we'll run an LLM pipeline for every given LLM and embed text using the provided LLM model.

Here's the heart of the script:
```python
LLM_MODEL_NAME = "model-name"
TEXT_EMBEDDING_PATH = "text-embedding-name"
# A text prompt to send through the LLM pipeline
LLM_PROMPT_INPUT = "Where is one place you'd go in Washington, DC?"
# The embeddings dataset is versatile! It plays with lists, datasets, or even generators.
EMBEDDING_DATA = [
    "US tops 5 million confirmed virus cases",
    "Canada's last fully intact ice shelf has suddenly collapsed, forming a Manhattan-sized iceberg",
    "Beijing mobilises invasion craft along coast as Taiwan tensions escalate"
]
# LLM PIPELINE
llm = LLM(LLM_MODEL_NAME, method="litellm")
print(llm([{"role": "user", "content": LLM_PROMPT_INPUT}]))
# TEXT EMBEDDING
embeddings = Embeddings(path=TEXT_EMBEDDING_PATH, method="litellm")
embeddings.index(EMBEDDING_DATA) # create an index for the text list
for query in ("feel good story", "climate change"): # now, let's embark on an embeddings search for each query
    # extract the uid of the first result
    # search result format: (uid, score)
    uid = embeddings.search(query, 1)[0][0]
    # print the text
    print("%-20s %s" % (query, EMBEDDING_DATA[uid]))
```

- <sup><sub>**friendly reminder**: _the library's author often [points out](https://github.com/neuml/txtai/pull/844#issuecomment-2563561232) that it's [not necessary](https://github.com/neuml/txtai/issues/843#issuecomment-2563244810) to explicitly pass the second argument `method='litellm'`. When you're learning something new, it's okay to avoid relying on shortcuts or "magic" until you're more comfortable. Once you understand the library better, you can start using these convenient features to your advantage. In this introduction, I'm intentionally including the second argument `method='litellm'` in the function. However, I'm choosing to leave it out in the Playground section_.</sub></sup>

## Playground

Once you've installed the necessary dependencies and configured the environment variables, you can play with and explore the code snippet. Enjoy your coding journey! 😊


In [64]:
# A text prompt to run through the LLM pipeline
# https://neuml.github.io/txtai/pipeline/text/llm/
LLM_PROMPT_INPUT = "Where is one place you'd go in Washington, DC?"
# The embeddings dataset is versatile! It plays with lists, datasets, or even generators.
# https://neuml.github.io/txtai/embeddings/
EMBEDDING_DATA = [
  "US tops 5 million confirmed virus cases",
  "Canada's last fully intact ice shelf has suddenly collapsed, forming a Manhattan-sized iceberg",
  "Beijing mobilises invasion craft along coast as Taiwan tensions escalate",
  "The National Park Service warns against sacrificing slower friends in a bear attack",
  "Maine man wins $1M from $25 lottery ticket",
  "Make huge profits without work, earn up to $100,000 a day"
]

for LLM_MODEL, LLM_CONFIG in LLM_MODELS.items():
  LLM_MODEL_NAME = LLM_CONFIG['LLM_MODEL_NAME']
  TEXT_EMBEDDING_PATH = LLM_CONFIG['TEXT_EMBEDDING_PATH']
  print("-" * 50)
  print(LLM_MODEL)

  # A quick note: you can skip specifying the `method` argument.
  # There's an autodetection logic designed to recognize it as a `litellm` model.
  # ```
  # llm = LLM(LLM_MODEL_NAME, method="litellm")
  # ```
  llm = LLM(LLM_MODEL_NAME)
  # # print(llm(LLM_PROMPT_INPUT, defaultrole="user"))
  print(llm([{"role": "user", "content": LLM_PROMPT_INPUT}]))

  embeddings = Embeddings(
    path=TEXT_EMBEDDING_PATH,
    # a quick note: you can skip specifying the `method` argument - there is autodetection logic
    # method="litellm"
  )
  # create an index for the list of text
  embeddings.index(EMBEDDING_DATA)
  print("." * 50)
  print("%-20s %s" % ("Query", "Best Match"))
  print("." * 50)
  # run an embeddings search for each query
  for query in ("feel good story", "climate change",
      "public health story", "war", "wildlife", "asia",
      "lucky", "dishonest junk"):
    # extract uid of first result
    # search result format: (uid, score)
    uid = embeddings.search(query, 1)[0][0]
    # print text
    print("%-20s %s" % (query, EMBEDDING_DATA[uid]))

print("-" * 50)

--------------------------------------------------
GEMINI
National Air and Space Museum
..................................................
Query                Best Match
..................................................
feel good story      Maine man wins $1M from $25 lottery ticket
climate change       Canada's last fully intact ice shelf has suddenly collapsed, forming a Manhattan-sized iceberg
public health story  US tops 5 million confirmed virus cases
war                  Beijing mobilises invasion craft along coast as Taiwan tensions escalate
wildlife             The National Park Service warns against sacrificing slower friends in a bear attack
asia                 Beijing mobilises invasion craft along coast as Taiwan tensions escalate
lucky                Maine man wins $1M from $25 lottery ticket
dishonest junk       Make huge profits without work, earn up to $100,000 a day
--------------------------------------------------
COHERE
As an AI chatbot, I do not have personal pr

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.10.output.LayerNorm.weight', 'encoder.layer.10.output.dense.bias', 'encoder.layer.10.output.dense.weight', 'encoder.layer.11.intermediate.dense.bias', 'encoder.layer.11.intermedi

..................................................
Query                Best Match
..................................................
feel good story      US tops 5 million confirmed virus cases
climate change       Make huge profits without work, earn up to $100,000 a day
public health story  Make huge profits without work, earn up to $100,000 a day
war                  US tops 5 million confirmed virus cases
wildlife             Maine man wins $1M from $25 lottery ticket
asia                 US tops 5 million confirmed virus cases
lucky                US tops 5 million confirmed virus cases
dishonest junk       Maine man wins $1M from $25 lottery ticket
--------------------------------------------------
OPENAI
One notable place to visit in Washington, DC is the National Mall. It's a central gathering place that features iconic landmarks such as the Lincoln Memorial, Washington Monument, and the U.S. Capitol. The expansive green space is not only historically significant but also offe